In [13]:
import numpy as np
import xarray as xr

In [18]:
import yaml
with open(f"config.yaml") as stream:
    try:
        CONFIG = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

In [20]:
if xgb is None:
        import xgboost as xgb

# Load model and weights:
model = xgb.XGBRegressor()
model.load_model(CONFIG["path_w_xgb_v1"])

In [30]:
inference_dts = np.arange(np.datetime64("2018-01-01T00"), np.datetime64("2022-01-01T00"), np.timedelta64(6,"h"))
inference_ilocs = [5284]
path_data = CONFIG["path_ec_euro"]
clim_feats = CONFIG["clim_feats"]
dyn_feats = CONFIG["dynamic_feats"]

In [27]:
ds = xr.open_zarr(path_data)
ds = ds.sel(time=inference_dts[0]).isel(x=inference_ilocs) #spatio-temporal selection
ds = ds.sel(clim_variable=clim_feats, variable=dyn_feats) #variable selection

In [39]:
ds

<xarray.Dataset> Size: 3kB
Dimensions:                     (x: 1, clim_variable: 23, variable: 33)
Coordinates:
  * clim_variable               (clim_variable) object 184B 'clim_cell_area' ...
    lat                         (x) float32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    lon                         (x) float32 4B dask.array<chunksize=(1,), meta=np.ndarray>
    time                        datetime64[ns] 8B 2018-01-01
  * variable                    (variable) object 264B 'met_ctpf' ... 'es'
  * x                           (x) int32 4B 72552
Data variables: (12/18)
    clim_data                   (x, clim_variable) float32 92B dask.array<chunksize=(1, 23), meta=np.ndarray>
    clim_maxs                   (clim_variable) float32 92B dask.array<chunksize=(23,), meta=np.ndarray>
    clim_means                  (clim_variable) float32 92B dask.array<chunksize=(23,), meta=np.ndarray>
    clim_mins                   (clim_variable) float32 92B dask.array<chunksize=(23,), meta=np.ndarray>
    clim_stdevs                 (clim_variable) float32 92B dask.array<chunksize=(23,), meta=np.ndarray>
    data                        (x, variable) float32 132B dask.array<chunksize=(1, 33), meta=np.ndarray>
    ...                          ...
    global_data_1stdiff_means   (x, variable) float32 132B dask.array<chunksize=(1, 33), meta=np.ndarray>
    global_data_1stdiff_stdevs  (x, variable) float32 132B dask.array<chunksize=(1, 33), meta=np.ndarray>
    global_data_maxs            (x, variable) float32 132B dask.array<chunksize=(1, 33), meta=np.ndarray>
    global_data_means           (x, variable) float32 132B dask.array<chunksize=(1, 33), meta=np.ndarray>
    global_data_mins            (x, variable) float32 132B dask.array<chunksize=(1, 33), meta=np.ndarray>
    global_data_stdevs          (x, variable) float32 132B dask.array<chunksize=(1, 33), meta=np.ndarray>

In [40]:
xr.merge([ds["clim_variable"], ds["variable"]])

ValueError: cannot create a Dataset from a DataArray with the same name as one of its coordinates

In [6]:
# Open the test data and select a single point in space to run the model on
ds_test = xr.open_zarr("/data/ecland_i6aj_o400_2010_2022_6h_euro.zarr").sel(time=slice("2018", "2022"))

clim_feats = (ds_test.sel(clim_variable=clim_feat_lst).clim_data
                 .expand_dims(time=ds_test.time)
                 .isel(x=5284, time=slice(0,-1))
                 .rename({"clim_variable": "variable"})
)
feats = ds_test.sel(variable=feat_lst).isel(x=5284, time=slice(0,-1)).data

feats = xr.concat((clim_feats, feats), dim="variable")

In [7]:
# Transform xr.Dataset into a np.ndarray
feats_arr = feats.values

In [12]:
%%time
# Loop over all times and iteratively apply the ML model to construct a 3-year ML estimate to the ec-land model
for x in range(len(feats_arr)-1):
    if x % 1000 == 0:
        print(f"on step {x}...")
    preds = model.predict(feats_arr[[x]])
    feats_arr[x+1, -len(targ_lst):] = np.clip(feats_arr[x, -len(targ_lst):] + preds, 0, None)

on step 0...
on step 1000...
on step 2000...
on step 3000...
on step 4000...
on step 5000...
on step 6000...
on step 7000...
CPU times: user 15 s, sys: 103 ms, total: 15.1 s
Wall time: 2 s
